In [28]:
from netCDF4 import Dataset
import numpy as np
from metpy.plots import wx_code_to_numeric
import pandas as pd

import metpy.calc as mpcalc
from metpy.units import units
from metpy.plots import current_weather, sky_cover, StationPlot, simple_layout
from metpy.io import metar
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [29]:
data = Dataset('./20221227_0900')
#for var in data.variables:
    #print(var)

In [31]:
station_name = data.variables['stationName'][:]
latitude = data.variables['latitude'][:]
longitude = data.variables['longitude'][:]
temp = data.variables['temperature'][:]
dwpt = data.variables['dewpoint'][:]
wspd = data.variables['windSpeed'][:]
wdir = data.variables['windDir'][:]
gust = data.variables['windGust'][:]
weather = data.variables['presWeather'][:]
visibility = data.variables['visibility'][:]
pslp = data.variables['seaLevelPress'][:]
skycover = data.variables['skyCover'][:]

In [32]:
stn_names = ['']*len(station_name)
for index in range(len(station_name)):
    stn = station_name[index].tobytes().decode().rstrip('\x00')
    stn_names[index] = stn
#print (stn_names)

cur_wx = ['']*len(weather)
for index in range(len(weather)):
    wx = weather[index].tobytes().decode().rstrip('\x00')
    cur_wx[index] = wx

wx_codes = wx_code_to_numeric(cur_wx)
#for i in range(len(wx_codes)):
    #print (cur_wx[i],wx_codes[i])

sky_cover_num = np.empty(len(skycover))
for i in range(len(skycover)):
    this_skycvr = skycover[i].tobytes().decode().rstrip('\x00')
    if this_skycvr[-3:] == 'SKC' or this_skycvr[-3:] == 'CLR':
        sky_cover_num[i] = 0
    elif this_skycvr[-3:] == 'FEW':
        sky_cover_num[i] = 2
    elif this_skycvr[-3:] == 'SCT':
        sky_cover_num[i] = 4
    elif this_skycvr[-3:] =='BRK':
        sky_cover_num[i] = 6
    elif this_skycvr[-3:] == 'OVC':
        sky_cover_num[i] = 8

In [33]:
wspd_from_nc = np.array(wspd)*units('m/s')
wdir_from_nc = np.array(wdir)*units('degrees')
u,v = mpcalc.wind_components(wspd_from_nc, wdir_from_nc)

/tmp/ipykernel_2203946/3146315831.py:3: UserWarning: Input over 12.566370614359172 radians. Ensure proper units are given.
  u,v = mpcalc.wind_components(wspd_from_nc, wdir_from_nc)


In [38]:
#convert data to pandas dataframe
data_for_df = {'station_id':stn_names,
               'latitude':latitude,
               'longitude':longitude,
               'wind_direction':wdir,
               'wind_speed':wspd,
               'wind_gust':gust,
               'visibility':visibility,
               'current_wx1_symbol':wx_codes,
               'air_temperature':temp,
               'dew_point_temperature':dwpt,
               'air_pressure_at_sea_level':pslp, 
               'skycover':sky_cover_num}

df = pd.DataFrame(data_for_df, index = stn_names)
#print(df)